In [1]:
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input, BatchNormalization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers, models, layers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications import ResNet50V2

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import io
from PIL import Image

In [2]:
from minio import Minio
from minio.error import ResponseError

minioClient = Minio(
                'minio-service.kubeflow:9000',
                access_key='minio', 
                secret_key='minio123', 
                secure=False
            )

In [3]:
INIT_LR = 1e-05  #1e-05  # This value is specific to what model is chosen: Inception, VGG or ResnNet.
EPOCHS = 30 
BS = 8

print("Loading images...")

data = []
labels = []

Loading images...


In [4]:

# read all X-Rays in the specified path, and resize them all to 256x256

for i in minioClient.list_objects('dataset', prefix='covid-19', recursive=True):
    label = i.object_name.split(os.path.sep)[-2]

    minioObj = minioClient.get_object('dataset', i.object_name)
    byteArray = minioObj.read()
    pil_image = Image.open(io.BytesIO(byteArray)).convert('RGB')
    image = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (256, 256))
    data.append(image)
    labels.append(label)

    
#normalise pixel values to real numbers between 0.0 - 1.0 
data = np.array(data) / 255.0
labels = np.array(labels)

# perform one-hot encoding for a 3-class labeling 
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels)
labels = to_categorical(integer_encoded)

print("... ... ", len(data), "images loaded in 3x classes:")
print(label_encoder.classes_)

... ...  227 images loaded in 3x classes:
['covid' 'normal' 'pneumonia_bac']


In [5]:
(x_train, x_val, y_train, y_val) = train_test_split(data, labels, test_size=0.20, stratify=labels)

In [ ]:
LEARNING_RATE = 0.00001 # List: 0.001, 0.0001, 0.0003, 0.00001, 0.00003
DENSE = 128             # Range: 50-200

In [6]:
model = Sequential()
adam_s = Adam(learning_rate = LEARNING_RATE)

#model.add(VGG16(input_shape=(224, 224, 3), include_top=False, weights='imagenet', pooling='average'))
model.add(ResNet50V2(input_shape=(256, 256, 3),include_top=False, weights='imagenet',pooling='average'))

for layer in model.layers:
    layer.trainable = False

model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(DENSE, activation='relu'))
model.add(Dense(DENSE, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=adam_s, metrics=['accuracy'])


model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Model)           (None, 8, 8, 2048)        23564800  
_________________________________________________________________
batch_normalization (BatchNo (None, 8, 8, 2048)        8192      
_________________________________________________________________
flatten (Flatten)            (None, 131072)            0         
_________________________________________________________________
dense (Dense)                (None, 128)               16777344  
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 387       
Total params: 40,367,235
Trainable params: 16,798,339
Non-trainable params: 23,568,896
___________________________________

In [7]:
# train the head of the network
print("Training the full stack model...")
H = model.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val), batch_size=8)

Training the full stack model...
Train on 181 samples, validate on 46 samples
Epoch 1/10
181/181 [==============================] - 21s 117ms/sample - loss: 0.8518 - accuracy: 0.5856 - val_loss: 1.3832 - val_accuracy: 0.4783
Epoch 2/10
181/181 [==============================] - 18s 98ms/sample - loss: 0.2696 - accuracy: 0.9503 - val_loss: 1.0109 - val_accuracy: 0.5870
Epoch 3/10
181/181 [==============================] - 18s 97ms/sample - loss: 0.1374 - accuracy: 0.9669 - val_loss: 0.8313 - val_accuracy: 0.6304
Epoch 4/10
181/181 [==============================] - 18s 97ms/sample - loss: 0.0665 - accuracy: 1.0000 - val_loss: 0.6971 - val_accuracy: 0.6957
Epoch 5/10
181/181 [==============================] - 17s 96ms/sample - loss: 0.0646 - accuracy: 0.9890 - val_loss: 0.5718 - val_accuracy: 0.8261
Epoch 6/10
181/181 [==============================] - 17s 97ms/sample - loss: 0.0342 - accuracy: 0.9945 - val_loss: 0.5134 - val_accuracy: 0.8696
Epoch 7/10
181/181 [=========================

In [9]:
import tensorflow

# model.save('resnet50.h5')
# new_model = tensorflow.keras.models.load_model('resnet50.h5')

# model.save('new-covid/1')

In [11]:
os.environ.update({
    'S3_ENDPOINT'          : 'minio-service.kubeflow:9000',
    'AWS_ACCESS_KEY_ID'    : 'minio',
    'AWS_SECRET_ACCESS_KEY': 'minio123',
    'S3_USE_HTTPS'         : '0',	# Whether or not to use HTTPS. Disable with 0.                        
    'S3_VERIFY_SSL'        : '0' 	# If HTTPS is used, controls if SSL should be enabled. Disable with 0.
})  

model.save("s3://model/new-covid/1")

INFO:tensorflow:Assets written to: s3://model/new-covid/1/assets


In [12]:
print("[INFO] loading untrained test images...")
imagePathTest = "./all/test"
imagePathsTest = list(paths.list_images(imagePathTest))
print(len(imagePathsTest))

dataTest = []
labelsTest = []

# loop over the image paths
for imagePathTest in imagePathsTest:

    # extract the class label from the filename
	  labelTest = imagePathTest.split(os.path.sep)[-2]
  
	  # load the image, swap color channels, and resize it to be a fixed
	  # 256x256 pixels while ignoring aspect ratio
	  imageTest = cv2.imread(imagePathTest)
	  imageTest = cv2.cvtColor(imageTest, cv2.COLOR_BGR2RGB)
	  imageTest = cv2.resize(imageTest, (256, 256))

	  # update the data and labels lists, respectively
	  dataTest.append(imageTest)
	  labelsTest.append(labelTest)
# convert the data and labels to NumPy arrays while scaling the pixel
# intensities to the range [0, 255]
dataTest = np.array(dataTest) / 255.0
labelsTest = np.array(labelsTest)


# perform one-hot encoding on the labels
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labelsTest)
labelsTest = to_categorical(integer_encoded)

print(labelsTest)

[INFO] loading untrained test images...
27
[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [14]:
predTest = model.predict(dataTest, batch_size=BS)
print(predTest)

[[0.0068557  0.02678966 0.96635467]
 [0.03875601 0.12253342 0.8387106 ]
 [0.01659377 0.42695478 0.55645144]
 [0.02627762 0.1159509  0.8577715 ]
 [0.00122558 0.00290923 0.9958652 ]
 [0.11798634 0.11365311 0.76836056]
 [0.00595036 0.00327872 0.99077094]
 [0.02789457 0.31944716 0.6526582 ]
 [0.00161389 0.00217918 0.9962069 ]
 [0.78979254 0.01098451 0.19922292]
 [0.2622419  0.1425396  0.5952185 ]
 [0.45421702 0.06200984 0.48377308]
 [0.4373251  0.01652702 0.5461478 ]
 [0.88145435 0.01188093 0.1066647 ]
 [0.9562477  0.00784262 0.03590965]
 [0.83745414 0.02574706 0.13679886]
 [0.48259628 0.08266492 0.43473884]
 [0.4833918  0.13619097 0.3804173 ]
 [0.00976312 0.92319435 0.06704257]
 [0.0161733  0.4871007  0.49672604]
 [0.01608484 0.58128566 0.40262946]
 [0.01657279 0.8673781  0.11604912]
 [0.02541882 0.61463773 0.35994342]
 [0.01231234 0.40142035 0.5862673 ]
 [0.015098   0.23081088 0.7540911 ]
 [0.00716045 0.91683143 0.0760081 ]
 [0.03639211 0.79298884 0.17061903]]


In [15]:
predClasses = predTest.argmax(axis=-1)
print(predClasses)

[2 2 2 2 2 2 2 2 2 0 2 2 2 0 0 0 0 0 1 2 1 1 1 2 2 1 1]


In [ ]:
!curl http://35.206.247.237:32380/v1/models/covid19

In [ ]:
!pip install tqdm
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess
import cv2
import json
import requests
from tqdm import tqdm




#get some sample data
sample_test_data = dataTest  #[0:2]
sample_test_labels = labelsTest #[0:2]

print(sample_test_data.shape)

class_names = ['covid', 'normal', 'pneumonia_bac']

# pre-process data 
#sample_test_data_processed = np.expand_dims(sample_test_data / 255., axis=3)

sample_test_data_processed = sample_test_data

# create payload
data = json.dumps({"signature_name": "serving_default", 
                   "instances": sample_test_data_processed.tolist()})

In [ ]:
with open("image_data.json", "w") as json_file:
    json.dump(data, json_file)
# print(data)

In [ ]:
!curl -v -H "Host: covid19.kubeflow.example.com" -X POST http://35.206.247.237:32380/v1/models/covid19:predict -d@image_data.json

In [ ]:
HEADERS = {'content-type': 'application/json'}
#MODEL1_API_URL = 'http://172.17.0.1:8511/v1/models/covid19/versions/2:predict'
MODEL1_API_URL = 'http://35.206.247.237:32380/v1/models/covid19:predict'

# inference request
json_response = requests.post(MODEL1_API_URL, data=data, headers=HEADERS)

print(json_response)

pred = json.loads(json_response.content.decode('utf-8'))
print(pred)

# view server response
predictions = json.loads(json_response.text)['predictions']
predictions = np.argmax(np.array(predictions), axis=1)
prediction_labels = [class_names[p] for p in predictions]

fig, ax = plt.subplots(6, 5, figsize=(60, 80))
for idx, img in enumerate(sample_test_data):
    rowidx = idx // 5
    colidx = idx % 5
    ax[rowidx, colidx].imshow(img)
    ax[rowidx, colidx].set_title('Actual: {}\nPredicted: {}'.format(class_names[sample_test_labels[idx].argmax(axis=-1)], 
                                                                    prediction_labels[idx]), fontsize=40)